# Calculating Housing Price Index for NYC from Transaction Data

## Setup and Prepare the dataframe

In this section we import the data and clean it up. If you trust that we've one everything correctly, feel free to skip to the next section, where we use the historical transaction data to construct the first housing price index.

In [1]:
!pip install geopandas
import pandas as pd
import geopandas as gpd
from pandas.api.types import CategoricalDtype
import numpy as np
from sklearn.linear_model import LinearRegression
from scipy import sparse

from matplotlib import pyplot as plt
from matplotlib.dates import date2num, num2date
from matplotlib import dates as mdates
from matplotlib import ticker
from matplotlib.colors import ListedColormap
from matplotlib.patches import Patch
import seaborn as sns
sns.set_theme()
%pdb

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
     |████████████████████████████████| 994 kB 4.0 MB/s eta 0:00:01
     |████████████████████████████████| 383 kB 42.9 MB/s eta 0:00:01
    ERROR: Command errored out with exit status 1:
     command: /opt/homebrew/opt/python@3.8/bin/python3.8 -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/private/var/folders/0p/y9v6h7xd3tgdp8cg8_5649n40000gn/T/pip-install-8dxelv5y/shapely_95d3327d41414fc88d4a2c3fee9d7a35/setup.py'"'"'; __file__='"'"'/private/var/folders/0p/y9v6h7xd3tgdp8cg8_5649n40000gn/T/pip-install-8dxelv5y/shapely_95d3327d41414fc88d4a2c3fee9d7a35/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');c

     |████████████████████████████████| 224 kB 103.3 MB/s eta 0:00:01
    ERROR: Command errored out with exit status 1:
     command: /opt/homebrew/opt/python@3.8/bin/python3.8 -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/private/var/folders/0p/y9v6h7xd3tgdp8cg8_5649n40000gn/T/pip-install-8dxelv5y/shapely_e8e6fb811f454f1a85bc5dc0a2424bbb/setup.py'"'"'; __file__='"'"'/private/var/folders/0p/y9v6h7xd3tgdp8cg8_5649n40000gn/T/pip-install-8dxelv5y/shapely_e8e6fb811f454f1a85bc5dc0a2424bbb/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base /private/var/folders/0p/y9v6h7xd3tgdp8cg8_5649n40000gn/T/pip-pip-egg-info-ptr8de1j
         cwd: /private/var/folders/0p/y9v6h7xd3tgdp8cg8_5649n40000gn/T/pip-install-8dxelv5y/shapely_e8e6fb811f454f

     |████████████████████████████████| 221 kB 116.1 MB/s eta 0:00:01
    ERROR: Command errored out with exit status 1:
     command: /opt/homebrew/opt/python@3.8/bin/python3.8 -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/private/var/folders/0p/y9v6h7xd3tgdp8cg8_5649n40000gn/T/pip-install-8dxelv5y/shapely_00f74e330f254ca4a4cebc18309bcfa2/setup.py'"'"'; __file__='"'"'/private/var/folders/0p/y9v6h7xd3tgdp8cg8_5649n40000gn/T/pip-install-8dxelv5y/shapely_00f74e330f254ca4a4cebc18309bcfa2/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base /private/var/folders/0p/y9v6h7xd3tgdp8cg8_5649n40000gn/T/pip-pip-egg-info-0_mq3ext
         cwd: /private/var/folders/0p/y9v6h7xd3tgdp8cg8_5649n40000gn/T/pip-install-8dxelv5y/shapely_00f74e330f254c

    ERROR: Command errored out with exit status 1:
     command: /opt/homebrew/opt/python@3.8/bin/python3.8 -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/private/var/folders/0p/y9v6h7xd3tgdp8cg8_5649n40000gn/T/pip-install-8dxelv5y/shapely_18dc760785d144409f3912d66fc08804/setup.py'"'"'; __file__='"'"'/private/var/folders/0p/y9v6h7xd3tgdp8cg8_5649n40000gn/T/pip-install-8dxelv5y/shapely_18dc760785d144409f3912d66fc08804/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base /private/var/folders/0p/y9v6h7xd3tgdp8cg8_5649n40000gn/T/pip-pip-egg-info-pdf7b4mv
         cwd: /private/var/folders/0p/y9v6h7xd3tgdp8cg8_5649n40000gn/T/pip-install-8dxelv5y/shapely_18dc760785d144409f3912d66fc08804/
    Complete output (17 lines):
    cannot open sh

     |████████████████████████████████| 179 kB 45.1 MB/s eta 0:00:01
    ERROR: Command errored out with exit status 1:
     command: /opt/homebrew/opt/python@3.8/bin/python3.8 -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/private/var/folders/0p/y9v6h7xd3tgdp8cg8_5649n40000gn/T/pip-install-8dxelv5y/shapely_9a5236f256c949e19328dd8c6bf2df03/setup.py'"'"'; __file__='"'"'/private/var/folders/0p/y9v6h7xd3tgdp8cg8_5649n40000gn/T/pip-install-8dxelv5y/shapely_9a5236f256c949e19328dd8c6bf2df03/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base /private/var/folders/0p/y9v6h7xd3tgdp8cg8_5649n40000gn/T/pip-pip-egg-info-zh13a7h2
         cwd: /private/var/folders/0p/y9v6h7xd3tgdp8cg8_5649n40000gn/T/pip-install-8dxelv5y/shapely_9a5236f256c949e

...

    Preparing wheel metadata ... done
  Using cached numpy-1.21.2-cp38-cp38-macosx_11_0_arm64.whl (12.3 MB)
  Using cached python_dateutil-2.8.2-py2.py3-none-any.whl (247 kB)
  Using cached six-1.16.0-py2.py3-none-any.whl (11 kB)
  Using cached attrs-21.2.0-py2.py3-none-any.whl (53 kB)
     |████████████████████████████████| 97 kB 34.8 MB/s eta 0:00:01
  Using cached click_plugins-1.1.1-py2.py3-none-any.whl (7.5 kB)
  ERROR: Command errored out with exit status 1:
   command: /opt/homebrew/opt/python@3.8/bin/python3.8 /opt/homebrew/lib/python3.8/site-packages/pip/_vendor/pep517/in_process/_in_process.py build_wheel /var/folders/0p/y9v6h7xd3tgdp8cg8_5649n40000gn/T/tmpbzfv8src
       cwd: /private/var/folders/0p/y9v6h7xd3tgdp8cg8_5649n40000gn/T/pip-install-8dxelv5y/pandas_72f851aba8c64e8d93a9b9ad519c7542
  Complete output (4887 lines):
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build/lib.macosx-11-arm64-3.8
  creating build/lib.macosx-11-arm64-3.

  clang -Wno-unused-result -Wsign-compare -Wunreachable-code -fno-common -dynamic -DNDEBUG -g -fwrapv -O3 -Wall -isysroot /Library/Developer/CommandLineTools/SDKs/MacOSX.sdk -DNPY_NO_DEPRECATED_API=0 -I/private/var/folders/0p/y9v6h7xd3tgdp8cg8_5649n40000gn/T/pip-build-env-6qv45h3a/overlay/lib/python3.8/site-packages/numpy/core/include -I/opt/homebrew/include -I/opt/homebrew/opt/openssl@1.1/include -I/opt/homebrew/opt/sqlite/include -I/opt/homebrew/opt/tcl-tk/include -I/opt/homebrew/Cellar/python@3.8/3.8.8_1/Frameworks/Python.framework/Versions/3.8/include/python3.8 -c pandas/_libs/tslibs/dtypes.c -o build/temp.macosx-11-arm64-3.8/pandas/_libs/tslibs/dtypes.o -Wno-error=deprecated-declarations -Wno-error=unreachable-code
  pandas/_libs/tslibs/dtypes.c:4174:3: warning: 'tp_print' is deprecated [-Wdeprecated-declarations]
    0, /*tp_print*/
    ^
  /opt/homebrew/Cellar/python@3.8/3.8.8_1/Frameworks/Python.framework/Versions/3.8/include/python3.8/cpython/object.h:260:5: note: 'tp_print' h

  Created wheel for pyproj: filename=pyproj-3.2.1-cp38-cp38-macosx_11_0_arm64.whl size=417527 sha256=925145eb2ae1c05e3e19db8f266eef91d4a7083c6dd9eb850b658c0dcd7f8b47
  Stored in directory: /Users/ilya/Library/Caches/pip/wheels/dc/00/b7/93aa3484cb42a70f15bd1a0a40d6996cd5b30d3859184b5123
  ERROR: Command errored out with exit status 1:
   command: /opt/homebrew/opt/python@3.8/bin/python3.8 -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/private/var/folders/0p/y9v6h7xd3tgdp8cg8_5649n40000gn/T/pip-install-8dxelv5y/fiona_8acf4851fbb64e5fa639eb8853e4a904/setup.py'"'"'; __file__='"'"'/private/var/folders/0p/y9v6h7xd3tgdp8cg8_5649n40000gn/T/pip-install-8dxelv5y/fiona_8acf4851fbb64e5fa639eb8853e4a904/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist

  Created wheel for shapely: filename=Shapely-1.5.9-py3-none-any.whl size=65455 sha256=687a0ab62bee0cbcdbf0fd9a3c0f915b219cfef3419a6750ef495c47e15f56ae
  Stored in directory: /Users/ilya/Library/Caches/pip/wheels/8a/bb/55/62b33bc28ee358f0f91547f0f29575142f099ee20c9f927ea2
Successfully built pyproj shapely
Failed to build pandas fiona
ERROR: Could not build wheels for pandas which use PEP 517 and cannot be installed directly


ModuleNotFoundError: No module named 'pandas'

In [ ]:
cols = ['BOROUGH', 'NEIGHBORHOOD', 'BUILDING CLASS CATEGORY',
       'TAX CLASS AT PRESENT', 'BLOCK', 'LOT', 'EASE-MENT',
       'BUILDING CLASS AT PRESENT', 'ADDRESS', 'APARTMENT NUMBER', 'ZIP CODE',
       'RESideNTIAL UNITS', 'COMMERCIAL UNITS', 'TOTAL UNITS',
       'LAND SQUARE FEET', 'GROSS SQUARE FEET', 'YEAR BUILT',
       'TAX CLASS AT TIME OF SALE', 'BUILDING CLASS AT TIME OF SALE',
       'SALE PRICE', 'SALE DATE']
cols = [col.lower().replace(" ","_") for col in cols]

Download all real estate transaction data from [NYC.gov](https://www1.nyc.gov/site/finance/taxes/property-annualized-sales-update.page) for January 2003 to July 2020. Only closed transactions are included. Column names are the same in all files. 

In [ ]:
# ex: https://www1.nyc.gov/assets/finance/downloads/pdf/rolling_sales/annualized-sales/2019/2019_manhattan.xlsx
url_prefix = 'https://www1.nyc.gov/assets/finance/downloads/pdf/rolling_sales'
years = range(2010,2020)
boroughs = ['manhattan', 'brooklyn', 'queens', 'bronx', 'statenisland']
links = []
for borough in boroughs:
  links += [f"{url_prefix}/annualized-sales/{year}/{year}_{borough}.xls{'x' if year > 2017 else ''}" for year in years ]
  links += [url_prefix + f"/rollingsales_{borough}.xlsx", url_prefix + f"/annualized-sales/2009_{borough}.xls",
          f"https://www1.nyc.gov/assets/finance/downloads/pdf/09pdf/rolling_sales/sales_2008_{borough}.xls", 
          f"https://www1.nyc.gov/assets/finance/downloads/excel/rolling_sales/sales_2007_{borough}.xls",
          *[f"https://www1.nyc.gov/assets/finance/downloads/sales_{borough if borough != 'statenisland' else 'si'}_0{n}.xls" for n in range(3,7)] ]


Exclude any transaction with sale price under \$100,000 as it might not be an arms length transaction or just bad data. The very last file we downloaded covers period of the last twelve months, so we need to remove duplicates in the overlapping period from the prior year.

Filter transactions to only include following categories:
* Coops
  * `09  coops - walkup apartments`
  * `10  coops - elevator apartments`
  * `17  condo coops`
* Condos
  * `12  condos - walkup apartments`
  * `13  condos - elevator apartments`

In [ ]:
df = pd.DataFrame()
for link in links:
    df = df.append(pd.read_excel(link, skiprows=4, names=cols, parse_dates=[20]), ignore_index=True)
    
df = df[['borough', 'neighborhood', 'building_class_category', 'block','lot', 'address', 'apartment_number', \
         'gross_square_feet', 'sale_price', 'year_built', 'sale_date']]
df.neighborhood = df.neighborhood.str.strip().str.title()
borough_type = CategoricalDtype(categories=['Manhattan', 'Bronx', 'Brooklyn', 'Queens', 'Staten Island'], ordered=True)
df['borough'] = df.borough.map({1:"Manhattan", 2: "Bronx", 3: "Brooklyn", 4: "Queens", 5: "Staten Island"}).astype(borough_type)

Read a GeoJSON file containing the geometry of every single block in New York. We only care about the center of each lot, so we discard the complex lot geometry. The file also contains lot areas which could be useful in future analysis.

In [ ]:
print("latest transaction in file:", df.sale_date.sort_values().iloc[-1])
SFH_CATEGORIES = "01"
COOP_CATEGORIES = "09|[^-]10|17" 
CONDO_CATEGORIES = "12|13"

sfh = df.building_class_category.str.contains(SFH_CATEGORIES)
coops = df.building_class_category.str.contains(COOP_CATEGORIES)
condos = df.building_class_category.str.contains(CONDO_CATEGORIES)

df['house_class'] = 'Condo'
df.loc[coops, 'house_class'] = 'Coop'
df.loc[sfh, 'house_class'] = 'SFH'

prop_type = CategoricalDtype(categories=['Condo', 'Coop', 'SFH', ], ordered=True)
df['house_class'] = df['house_class'].astype(prop_type)

df = df[coops | condos | sfh]
df = df[(df.sale_price > 100000)].drop_duplicates()

Combine lat/long coordinates of every block with the transaction dataset

In [ ]:
#df = df.set_index(['borough', 'block']).join(lotblock).reset_index()

Calculate the center point of each neighborhood

We need to create a unique ID to track unique houses. For condos, that is the block/lot number combination. But all Coops in the same building share the exact same lot number, so it's not a unique ID. We extract the apartment number from the ending of the address.

In [ ]:
df.loc[condos | sfh, "uid"] = df[condos | sfh].block.map(str) + '_' + df[condos | sfh].lot.map(str)
df.loc[coops, "uid"] = df[coops].block.map(str) + '_' + \
               df[coops].address.str.rsplit(',',n=1,expand=True)[1] \
                                .str.lower() \
                                .str.replace('(?:unit)?(?:apt)?[^a-z0-9\n]', '', regex=True)

We convert transaction periods to quarters (our index will be quarterly as a result). If the same house sold multiple times in the same quarter, remove the first transaction(s) and only keep the last transaction. This most likely represents a contractor flip.

In [ ]:
df['period'] = pd.PeriodIndex(df.sale_date, freq='Q')
df = df.drop_duplicates(subset=['period','uid'],keep='last')
df.groupby('period').house_class.value_counts().unstack().plot(kind='line', stacked=True,title="Stacked Transactions by Type")

In [ ]:
df[(df.borough == "Manhattan") & (df.block==886) & (df.apartment_number=='1B')].sort_values('sale_date')

In [ ]:
lotblock = gpd.read_file('https://drive.google.com/uc?export=download&id=1T12uhUyAwBdwZ4EKzAjXb3X1PiY3RBIp')
lotblock = lotblock[['BORO', 'BLOCK', 'SHAPE_AREA', 'geometry']].dropna()
lotblock.BORO = lotblock.BORO.astype(int).map({1:"Manhattan", 2: "Bronx", 3: "Brooklyn", 4: "Queens", 5: "Staten Island"}).astype(borough_type)
lotblock.BLOCK = lotblock.BLOCK.astype(int)
lotblock.columns = ['borough', 'block', 'area', 'geometry']
lotblock = lotblock.set_index(['borough', 'block']).to_crs(epsg=3395)
lotblock = lotblock.join(df.set_index(['borough', 'block'])[['neighborhood', 'sale_price', 'sale_date']]).dropna()
hoods = lotblock.dissolve(by=['borough', 'neighborhood'], aggfunc='count').set_crs(epsg=3395)
hoods.plot(column='sale_price', linewidth=0, legend=True)

Plot median sale prices across all of New York (hard to tell differences as Manhattan is so much more expensive)

Plot the median sale price of each neighborhood in Manhattan

In [ ]:
med_px.loc['Manhattan'].plot(column="sale_price", linewidth=0, legend=True)


This loop iterates through every borough and combines neighborhoods into top 10 largest. It does this by taking the smallest neighborhood (by # of transaction) and merging it with geographically closest neighborhood (by centroid). Then it repeats until only 10 neighborhoods are left.

In [ ]:
hood_dict = {}
for boro in hoods.index.levels[0]:
  print("Merging neighborhoods in", boro)
  while len(hoods.loc[boro].index) > 10:
    smallest = hoods.loc[boro].sale_date.sort_values(ascending=True).index[0]
    smallest_center = hoods.loc[boro].centroid.loc[smallest]
    nearest_hood = hoods.loc[boro].centroid.distance(smallest_center).sort_values().index[1]
    hood_dict[smallest] = nearest_hood
    for key, value in hood_dict.items():
      if value == smallest:
        hood_dict[key] = nearest_hood
    old_cnt = hoods.loc[(boro, nearest_hood), 'sale_date']
    poly = hoods.loc[boro].loc[[nearest_hood, smallest]].unary_union
    hoods.loc[(boro, nearest_hood), 'geometry'] = hoods.loc[(boro, nearest_hood), 'geometry'] =gpd.GeoDataFrame(geometry=[poly]).geometry.values
    hoods.loc[(boro, nearest_hood), 'sale_date'] += hoods.loc[(boro, smallest), 'sale_date']
    new_cnt = hoods.loc[(boro, nearest_hood), 'sale_date']
    hoods.drop(smallest, level='neighborhood', inplace=True)

In [ ]:
hoods.plot(column='sale_date', linewidth=0, legend=True, figsize=(20,10))

In [ ]:
lotblock.replace({'neighborhood': hoods_dict})
med_px = lotblock[lotblock.sale_date >= "2020-08-31"].dissolve(by=['borough', 'neighborhood'], aggfunc='median')
med_px.geometry = med_px.convex_hull
med_px.plot(column="sale_price", linewidth=0, legend=True)

In [ ]:
qtr_offset = -1  # change this to -1 for latest quarter, -2 for the one before that and so on
mr_period = np.sort(df.period.unique())[qtr_offset]
print("Most recent period is", mr_period, "and # of transaction in this period:")
df[df.period == mr_period].house_class.value_counts()

In [ ]:
''' Function takes in DataFrame rpt_sales with columns `uid`, `period`, and `sale_price`. 
    Returns:
        design matrices X, W, and Z as specified in the Case Shiller index methodology
        array deltaT:    number of periods elapsed between each sale pair
        array uid_key:   for each row of matrices, which house it represents
        array periods:   list of all unique periods, so columns can be converted to Periods 
'''
def createDesignMatrices(rpt_sales):
    grouped = rpt_sales.groupby('uid')
    S = np.sum(grouped.size()-1)
    periods = list(rpt_sales.period.unique())
    T = len(periods)

    # construct blank design matrices
    X = np.zeros((S, T))
    deltaT = np.zeros((S,1))
    
    uid_key = []
    row = 0
    for house, group in grouped:
      for i in range(len(group)-1):
        first = (group.iloc[i].sale_price, periods.index(group.iloc[i].period))
        second = (group.iloc[i+1].sale_price, periods.index(group.iloc[i+1].period))

        X[row,first[1]] = -first[0]
        X[row,second[1]] = second[0]
        deltaT[row] = second[1] - first[1]

        uid_key.append(house)
        row += 1

    W = np.reshape(-X[:,0],(-1,1))
    X = X[:,1:]
    Z = np.sign(X)

    return sparse.csr_matrix(X), W, sparse.csr_matrix(Z), deltaT, uid_key, periods

'''
    Constructs Case Shiller repeat sales index
    inputs: X,Z,W - design matrices as specified in the Case Shiller methodology. X and Z are sparse matrices
            periods - array of all periods in the sample
    outputs: DataFrame with index of periods and columns `weighted` and `unweighted` for the house price index
            first period index value is guaranteed to be 1.0 by convention
'''
def buildIndex(X, Z, W, deltaT, periods):
    #1) Estimate b by running regression using instrumental variables b = (Z'X)^-1 x X'w
    coeff = sparse.linalg.inv(Z.T * X) * Z.T * W
    #coeff = np.linalg.inv(Z.T.dot(X)).dot(Z.T).dot(W)
    idx = 1/(coeff.reshape(X.shape[1]))
    unweighted = pd.DataFrame([1,*idx], index=periods, columns=['unweighted'])

    #2) Calculate weights for each observation 
    resid = W - X.dot(coeff)
    reg = LinearRegression().fit(deltaT, resid**2)
    weights = np.sqrt(reg.predict(deltaT).clip(min=0.00001))
    Omega = sparse.diags(np.reshape(1/weights,X.shape[0]))

    #3) Estimate b again incorporating weights b = (Z'OmegaX)^-1 x Z'Omega w
    coeff_weighted = sparse.linalg.inv(Z.T * Omega * X) * Z.T * Omega * W
    idx_weighted = 1/(coeff_weighted.reshape(X.shape[1]))
    weighted = pd.DataFrame([1,*idx_weighted],index=periods, columns=['weighted'])

    return pd.concat([unweighted, weighted], axis=1), weights

'''
    Function generates unweighted, weighted, and smoothed (rolling) values for price index
    Takes in DataFrame and column key, an index will be returned for each unique value in df.key
'''
def generateIndexValues(df, key):
    grouped = df.groupby(key)
    results = None
    for name, group in grouped:
        X, W, Z, deltaT, uid_key, periods = createDesignMatrices(group)
        try:
            idx,weights = buildIndex(X,Z,W, deltaT, periods)
            idx[key] = name
            
            smoothed = idx.weighted.rolling(4,
                win_type='gaussian',
                min_periods=1,
                center=True).mean(std=2).rename("smoothed")
            
            idx = pd.concat([idx,smoothed], axis=1)
            
            if results is None:
                results = idx
            else:
                results = pd.concat([results, idx])
            
        except Exception as e:
            print(name, e)
      
    return results

def plot_standings(data, title, figsize=(15,2.5)):
    fig, ax = plt.subplots(figsize=figsize)
    ax.set_title(title)
    bars = ax.bar(data.index, data, width=.825, capsize=2, color=[.7,.7,.7])

    ax.axhline(0.0, linestyle=':', color='k', lw=1)

    ax.yaxis.set_major_formatter(ticker.StrMethodFormatter("{x:.1%}"))

    ax.spines['top'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.spines['right'].set_visible(False)

    ax.set_xticklabels(data.index, rotation=90, fontsize=11)
    fig.set_facecolor('w')

''' Helper function to create a time series index plot. 
Result is a dataframe with cols weighted and smoothed'''
def plot_idx(result, ax, area_name, ylim=(0.5,3.0)):
    index = result.index.to_timestamp()
    ax.set_title(f"{area_name} House Price Index")

    ax.plot(index, result.weighted.values, 
                   c='k',
                   linestyle=':',
                   alpha=.5, label="Actual")

    ax.plot(index, result.smoothed.values, label="Smoothed")
    
    ax.axhline(1.0, c='k', lw=1, alpha=.25)
    ax.legend(frameon=False)
    ax.grid(which='major', axis='y', c='k', alpha=.1, zorder=-2)
    
    ax.spines['top'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.spines['right'].set_visible(False)
    
    ax.yaxis.set_major_locator(ticker.MultipleLocator(0.5))
    ax.yaxis.set_major_formatter(ticker.StrMethodFormatter("{x:.1f}"))
    
    ax.set_ylim(ylim[0],ylim[1])
    ax.margins(0)

## Calculate the index values for NYC

Filter for only homes that have sold more than once (not technically required but reduces memory usage), sort by date

In [ ]:
rpt_sales = df[['period', 'uid', 'sale_price', 'borough', 'neighborhood', 'sale_date', 'house_class']] \
                .groupby('uid') \
                .filter(lambda x: len(x) >= 2) \
                .sort_values('sale_date')

rpt_sales.groupby('period').house_class.value_counts().unstack().plot(kind='line', stacked=True,title="Repeat Transactions Only by Type")

Number of repeat transactions in the most recent period

In [ ]:
rpt_sales[rpt_sales.period == mr_period].house_class.value_counts()

Build design matrices, and construct the index values. Indices are constructed using the repeat sales methodology, meaning that we only look at homes that have sold several times and track the percentage change in price between the first sale and the next sale, which ideally represents an apples-to-apples comparison. See Case Shiller methodology for full description. Known issues is that if the house was remodeled or otherwise improved, we don't have this information so it wouldn't get incorporated. But homes also on average lose value to depreciation/age, and with thousands of transactions this represents the best price comparison possible.

In [ ]:
X, W, Z, deltaT, uid_key, periods = createDesignMatrices(rpt_sales)
idx_all, weights = buildIndex(X, Z, W, deltaT, periods)

Plot the weighted house price index along with a 1-year/4-quarter rolling average.

In [ ]:
smoothed = idx_all.weighted.rolling(4,
        win_type='gaussian',
        min_periods=1,
        center=True).mean(std=2).rename("smoothed")

fig, ax = plt.subplots(figsize=(600/72,400/72))
idx_all = pd.concat([idx_all, smoothed], axis=1)
plot_idx(idx_all, ax, "NYC")

# Borough

Create a separate index for each borough, like Manhattan and Brooklyn

In [ ]:
borough_idx = generateIndexValues(rpt_sales,'borough')

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(15, 6), sharey=True)

plot_idx(idx_all, axes.flat[0], "All", ylim=(1,3.5))

for row, (name, idx) in enumerate(borough_idx.groupby('borough'),1):
    plot_idx(idx, axes.flat[row], name, ylim=(1,3.5))
                                  
fig.tight_layout()

In [ ]:
mr = borough_idx.set_index('borough',append=True).smoothed.unstack('borough').pct_change(4).iloc[qtr_offset]
mr.loc['All'] = idx_all.smoothed.pct_change(4).iloc[qtr_offset:].values[0]
plot_standings(mr, title=f"Change in Housing Prices, {idx_all.index[qtr_offset]} YoY", figsize=(5,2.5))
mr.map("{:,.0%}".format)

In [ ]:
mr = borough_idx.set_index('borough',append=True).smoothed.unstack('borough').pct_change().iloc[qtr_offset]
mr.loc['All'] = idx_all.smoothed.pct_change().iloc[qtr_offset:].values[0]
plot_standings(mr, title=f"Change in Housing Prices, {idx_all.index[qtr_offset]} QoQ", figsize=(5,2.5))
mr.map("{:,.0%}".format)

## Calculate index values for SFH, Condos, and Coops

We use the same methodology to generate separate indexes for Condos and Coops in Manhattan. As shown above, there are 4x more condo transactions than coop transactions in our dataset, so the condo index has higher statistical validity. 

In [ ]:
house_class_idx = generateIndexValues(rpt_sales,'house_class')

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=4, figsize=(15, 6), sharey=True)

plot_idx(idx_all, axes.flat[0], "All", ylim=(1,3.5))

for row, (name, idx) in enumerate(house_class_idx.groupby('house_class'),1):
    plot_idx(idx, axes.flat[row], name, ylim=(1,3.5))
                                  
fig.tight_layout()

Show the year over year change in prices:

In [ ]:
mr = house_class_idx.set_index('house_class',append=True).smoothed.unstack('house_class').pct_change(4).iloc[qtr_offset]
mr.loc['All'] = idx_all.smoothed.pct_change(4).iloc[qtr_offset:].values[0]
plot_standings(mr, title=f"Change in Housing Prices, {idx_all.index[qtr_offset]} YoY", figsize=(5,2.5))
mr.map("{:,.0%}".format)

Show the Quarter over Quarter change:

In [ ]:
mr = house_class_idx.set_index('house_class',append=True).smoothed.unstack('house_class').pct_change().iloc[qtr_offset]
mr.loc['All'] = idx_all.smoothed.pct_change().iloc[qtr_offset:].values[0]
plot_standings(mr, title=f"Change in Housing Prices, {idx_all.index[qtr_offset]} QoQ", figsize=(5,2.5))
mr.map("{:,.0%}".format)

## Calculate index values for each neighborhood

Combine some of the smaller neighborhoods together and eliminate very tiny neighborhoods. 

Here are the reasons why certain neighborhoods were combined:
* All UES and UWS subdivions merged together as some were too few or generating volatile values
* Chinatown and Little Italy both had way too few data transactions on their own. Lower East Side was generating too volatile index values and is adjancent to both - merged together
* Javits Center (now known as "Hudson Yards") only had 119 datapoints in last 10 years. Midtown CBD has very few transactions as well. Fashion district and Gramercy were generating volatile index values. All three merged together as Midtown South.
* Southbridge and Civic Center are tiny neighborhoods below Tribeca with few transactions, merged with Financial District.
* Harlem, East, West and Upper had too few transactions and generated volatile values on their own, merged together
* Kips Bay was generating very volatile index values. Kips Bay merged with Flatiron (I debated merging with Murray Hill or Gramercy). 
* Manhattan Valley, Morningside, and Inwood merged together as too few values

Brooklyn has a LOT of different neighborhoods and much lower transaction volume per neighborhood. Many need to be removed or grouped to generate valid results.
* Marine Park, Bush Terminal, and Navy Yard have zero datapoints and must be excluded (otherwise model returns infinite values)
* Red Hook is a singular matrix due to very few listings, must be classified with others



In [ ]:
# exclude neighborhoods with very few datapoints
FILTERED_NEIGHBORHOODS=['Manhattan-Unknown',
                        'Brooklyn-Unknown',
                        'Roosevelt Island',
                        'Bush Terminal',
                        'Navy Yard',
                        'Staten Island Unknown',
                        ]
COMBINED_NEIGHBORHOODS = {
    'Chinatown':                 'Chinatown/Little Italy/LES',
    'Little Italy':              'Chinatown/Little Italy/LES',
    'Lower East Side':           'Chinatown/Little Italy/LES',
    'East Village':              'East Village/Alphabet City',
    'Alphabet City':             'East Village/Alphabet City',
    'Civic Center':              'Financial',
    'Southbridge':               'Financial',
    'Kips Bay':                  'Flatiron/Kips Bay',
    'Flatiron':                  'Flatiron/Kips Bay',
    'Greenwich Village-West':    'Greenwich Village',
    'Greenwich Village-Central': 'Greenwich Village',
    'Harlem-West':               'Harlem',
    'Harlem-Upper':              'Harlem',
    'Harlem-East':               'Harlem',
    'Harlem-Central':            'Harlem',
    'Javits Center':             'Midtown South',
    'Fashion':                   'Midtown South',
    'Midtown Cbd':               'Midtown South',
    'Gramercy':                  'Midtown South',
    'Murray Hill':               'Midtown South',
    'Upper East Side (96-110)':  'Upper East Side',
    'Upper East Side (79-96)':   'Upper East Side',
    'Upper East Side (59-79)':   'Upper East Side',
    'Upper West Side (59-79)':   'Upper West Side',
    'Upper West Side (79-96)':   'Upper West Side',
    'Upper West Side (96-116)':  'Upper West Side',
    'Manhattan Valley':          'Washington Heights',
    'Morningside Heights':       'Washington Heights',
    'Inwood':                    'Washington Heights',
    'Washington Heights Upper':  'Washington Heights',
    'Washington Heights Lower':  'Washington Heights',
    'Manhattan Beach':           'Brooklyn South',
    'Gowanus':                   'Brooklyn Other',
    'Marine Park':               'Brooklyn South',
    'Bush Terminal':             'Brooklyn Other',
    'Flatbush-Lefferts Garden':  'Flatbush',
    'Navy Yard':                 'Brooklyn Other',
    'Ocean Parkway-South':       'Brooklyn Other',
    'Ocean Parkway-North':       'Brooklyn Other',
    'Wyckoff Heights':           'Brooklyn Other',
    'Old Mill Basin':            'Brooklyn Other',
    'Ocean Hill':                'Brooklyn Other',
    'Bushwick':                  'Brooklyn Other',
    'Midwood':                   'Brooklyn Other',
    'Canarsie':                  'Brooklyn Other',
    'Bath Beach':                'Bay Ridge',
    'Bensonhurst':               'Bay Ridge',
    'Dyker Heights':             'Bay Ridge',
    'Borough Park':              'Bay Ridge',
    'Kensington':                'Brooklyn Other',
    'Coney Island':              'Brooklyn South',
    'Red Hook':                  'Brooklyn Other',
    'Manhattan Beach':           'Brooklyn South',
    'Sheepshead Bay':            'Brooklyn South',
    'Bergen Beach':              'Brooklyn South',
    'Madison':                   'Brooklyn South',
    'Gravesend':                 'Brooklyn South',
    'Marine Park':               'Brooklyn South',
    'Brighton Beach':            'Brooklyn South',
    'Flatbush-East':             'Flatbush',
    'Flatbush-Central':          'Flatbush',
    'Flatbush-North':            'Flatbush',
    'Flatbush-Lefferts Garden':  'Flatbush',
    'Williamsburg-Central':      'Williamsburg',
    'Williamsburg-East':         'Williamsburg',
    'Williamsburg-North':        'Williamsburg',
    'Williamsburg-South':        'Williamsburg',
    'Cobble Hill-West':          'Brooklyn-Downtown',
    'Park Slope South':          'Park Slope',
    'Windsor Terrace':           'Park Slope',
    'Downtown-Fulton Mall':      'Brooklyn-Downtown',
    'Downtown-Metrotech':        'Brooklyn-Downtown',
    'Downtown-Fulton Ferry':     'Brooklyn-Downtown',
    'Boerum Hill':               'Brooklyn-Downtown',
    'Clinton Hill':              'Brooklyn-Downtown',
    'Cobble Hill':               'Brooklyn-Downtown',
    'Brooklyn Heights':          'Brooklyn-Downtown',
    'Gowanus':                   'Brooklyn-Downtown',
}
filtered = rpt_sales.neighborhood.isin(FILTERED_NEIGHBORHOODS)
neighborhood_sales = rpt_sales[~filtered].replace({"neighborhood": hoods_dict})
neighborhood_sales.groupby('neighborhood').size().sort_values().plot.bar(figsize=(15,3), title="Number of repeat sales in dataset")

Show the number of repeat transactions in each neighborhood for the most recent period.

In [ ]:
neighborhood_sales[neighborhood_sales.period==mr_period].groupby('neighborhood').size().sort_values(ascending=False)

Create index values for every neighborhood, along with rolling averages

In [ ]:
neighborhood_idx = generateIndexValues(neighborhood_sales,'neighborhood')

Display graph indices for all neighborhoods, in alphabetical order

In [ ]:
ncols = 3
nrows = int(np.ceil(len(neighborhood_idx.groupby('neighborhood').groups) / ncols))

fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(15, nrows*3), sharey=False)

for row, (name, idx) in enumerate(neighborhood_idx.groupby('neighborhood')):
    ylim = (0, max(4.0, idx.smoothed.max()))
    plot_idx(idx, axes.flat[row], name, ylim=ylim)
                                  
fig.tight_layout()

Calculate the year over year price change for each neighborhood, sort values, and plot:

In [ ]:
mr = neighborhood_idx.set_index('neighborhood',append=True).smoothed.unstack('neighborhood').pct_change(4).iloc[qtr_offset]
mr.sort_values(inplace=True)
plot_standings(mr, title=f"Change in Housing Prices, {neighborhood_idx.index[qtr_offset]} YoY")

Print the percent change in house values as well for reference:

In [ ]:
mr.map('{:,.0%}'.format)

Show the Quarter over Quarter change:

In [ ]:
mr = neighborhood_idx.set_index('neighborhood',append=True).smoothed.unstack('neighborhood').pct_change().iloc[qtr_offset]
mr.sort_values(inplace=True)
plot_standings(mr, title=f"Change in Housing Prices, {neighborhood_idx.index[qtr_offset]} QoQ")

In [ ]:
mr.map('{:,.0%}'.format)